In [25]:
!pip install python-dotenv

In [4]:
!pip install pymongo pandas

In [26]:
%load_ext dotenv
%dotenv

In [29]:
from pymongo import MongoClient
import pandas as pd  
import os 

client = MongoClient(os.getenv("MONGODB_URI") ) 


db = client['job_matching_db']  


print("My Collections :", db.list_collection_names())


My Collections : ['candidates', 'jobs']


In [37]:
jobs_collection = db['jobs']
jobs_all = list(jobs_collection.find())  
jobs_df = pd.DataFrame(jobs_all)
jobs_df.head() 

 number of jobs : 25
1er job : {'_id': 'job_001', 'title': 'Backend Developer', 'required_skills': ['Node.js', 'MongoDB', 'REST'], 'experience_required': 2, 'location': 'Remote', 'description': 'Backend developer needed to build REST APIs using Node.js and MongoDB.', 'search_text': 'Backend Developer\nRemote\nNode.js MongoDB REST \nExperience: 2 ans\nBackend developer needed to build REST APIs using Node.js and MongoDB.'}


,_id,title,required_skills,experience_required,location,description,search_text
0,job_001,Backend Developer,"[Node.js, MongoDB, REST]",2,Remote,Backend developer needed to build REST APIs us...,Backend Developer\nRemote\nNode.js MongoDB RES...
1,job_002,Frontend Developer,"[React, JavaScript, CSS]",1,Hybrid,Frontend developer to build responsive user in...,Frontend Developer\nHybrid\nReact JavaScript C...
2,job_003,Full Stack Engineer,"[Node.js, React, PostgreSQL]",3,Remote,Looking for a full stack engineer comfortable ...,Full Stack Engineer\nRemote\nNode.js React Pos...
3,job_004,Data Analyst,"[Python, SQL, Data Visualization]",2,Onsite,Analyze business data and create dashboards us...,Data Analyst\nOnsite\nPython SQL Data Visualiz...
4,job_005,Machine Learning Engineer,"[Python, Machine Learning, Pandas]",3,Remote,Build and deploy machine learning models for r...,Machine Learning Engineer\nRemote\nPython Mach...


In [32]:
jobs_df.shape

(25, 7)

In [36]:
candidates_collection = db.candidates
candidates_all = candidates_collection.find()
candidates_df = pd.DataFrame(candidates_all)
candidates_df.head()

,_id,name,title,experience_years,skills,education,summary,search_text
0,cand_001,Ahmed Ben Ali,Backend Developer,3,"[Node.js, MongoDB, REST]",Computer Science,Backend developer with experience building API...,Ahmed Ben Ali Backend Developer Node.js MongoD...
1,cand_002,Sara Trabelsi,Frontend Developer,2,"[React, JavaScript, CSS]",Software Engineering,Frontend developer focused on responsive and a...,Sara Trabelsi Frontend Developer React JavaScr...
2,cand_003,Youssef Kacem,Full Stack Developer,4,"[React, Node.js, PostgreSQL]",Computer Engineering,Full stack engineer experienced in building sc...,Youssef Kacem Full Stack Developer React Node....
3,cand_004,Amira Zribi,Data Analyst,2,"[Python, SQL, Power BI]",Data Science,Data analyst skilled in extracting insights fr...,Amira Zribi Data Analyst Python SQL Power BI ...
4,cand_005,Mohamed Ghali,Machine Learning Engineer,3,"[Python, Machine Learning, Scikit-learn]",AI Engineering,ML engineer with experience building predictiv...,Mohamed Ghali Machine Learning Engineer Python...


In [38]:
candidates_df.shape


(20, 8)